In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Do reconstruction and check discrepancy

### 1 Check that everything works on an example setup

In [ ]:
from angle_set import AngleSet
from pylocus.algorithms import procrustes
from algorithms import reconstruct_from_angles

d = 2 
N = 5
np.random.seed(51)
angle_set = AngleSet(N=N, d=d)
angle_set.set_points(mode='random')
angle_set.plot_all()

points = reconstruct_from_angles(angle_set.theta_tensor)
points_fitted, *_ = procrustes(angle_set.points, points, scale=True)

plt.figure()
plt.scatter(*points_fitted.T, label='fitted')
plt.scatter(*angle_set.points.T, label='original', marker='x')
plt.axis('equal')
plt.legend(loc='best')

In [ ]:
from pylocus.basics_angles import get_theta_tensor
from pylocus.basics import projection
from simulation_discrepancy import get_noisy
from angle_set import get_linear_constraints

theta_noisy = get_noisy(angle_set.theta)

# reconstruct raw
theta_noisy_tensor = get_theta_tensor(theta_noisy, angle_set.corners, angle_set.N)

points_noisy_unfit = reconstruct_from_angles(theta_noisy_tensor)
points_noisy, *_ = procrustes(angle_set.points, points_noisy_unfit, scale=True)

# impose linear constraints
Afull, bfull = get_linear_constraints(angle_set)
bfull = bfull.flatten()
theta_linear, __, __ = projection(theta_noisy, Afull, bfull)
theta_linear_tensor = get_theta_tensor(theta_linear, angle_set.corners, angle_set.N)

points_linear_unfit = reconstruct_from_angles(theta_linear_tensor)
points_linear, *_ = procrustes(angle_set.points, points_linear_unfit, scale=True)

# plot results
plt.figure()
plt.scatter(*points_linear.T, label='linear')
plt.scatter(*points_noisy.T, label='noisy')
plt.scatter(*angle_set.points.T, label='original', marker='x')
plt.axis('equal')
plt.legend(loc='best')

### 2 Find out which constraints imply other constraints

In [ ]:
from algorithms import solve_constrained_optimization, constraint_sine_multi

# choose which sine constraints to include.
if N==4:
    choices_sine = range(1)
elif N==5: 
    choices_sine = range(3)
elif N==6: 
    choices_sine = range(6) # range(7) 

print('minimized:')
theta_sine, success = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                            Afull=Afull,
                                            bfull=bfull, N=N,
                                            choices_sine=choices_sine)
[print(constraint_sine_multi(theta_sine, angle_set.corners, N, [choice])) for choice in choices_sine]

print('not minimized:')
n_available = angle_set.get_n_sine()
other = list(range(n_available))
[other.remove(choice) for choice in choices_sine]
for o in other:
    print(constraint_sine_multi(theta_sine, angle_set.corners, N, [o]))

In [ ]:
d=2
np.random.seed(1)
for N in range(4, 8):
    print('===== N={} ====='.format(N))
    angle_set = AngleSet(N=N, d=d)
    angle_set.set_points('random')
    Atri, btri = angle_set.get_triangle_constraints()
    Aray, bray = angle_set.get_ray_constraints()
    Alinear = np.vstack((Aray, Atri))
    blinear = np.hstack((bray, btri))
    print('considering only the minimal constraints:')
    print('shape:', Alinear.shape, 'rank:', np.linalg.matrix_rank(Alinear))

    Apoly, bpoly = angle_set.get_polygon_constraints(range(3, N))
    Alinear = np.vstack((Aray, Apoly))
    blinear = np.hstack((bray, bpoly))
    print('considering more constraints:')
    print('shape:', Alinear.shape, 'rank:', np.linalg.matrix_rank(Alinear))

### 3 Calculate discrepancy

In [ ]:
from angle_set import create_theta
from simulation_discrepancy import mae
from algorithms import reconstruct_theta

N=4
d=2
np.random.seed(1)
angle_set = AngleSet(N=N, d=d)
angle_set.set_points('random')


num_sine = angle_set.get_n_sine()
num_linear = angle_set.get_n_linear()
choices_sine = range(num_sine)
choices_linear = range(num_linear)

# denoise
theta_noisy = get_noisy(angle_set.theta)
Afull, bfull = get_linear_constraints(angle_set)

theta_sine, __ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                            Afull=Afull, bfull=bfull, N=N,
                                            choices_sine=choices_sine, 
                                            choices_linear=choices_linear)

# reconstruct
theta_sine_reconstructed, points_sine = reconstruct_theta(theta_sine, angle_set.corners, angle_set.N)

mae_noisy = mae(angle_set.theta, theta_noisy)
mae_denoised = mae(angle_set.theta, theta_sine)
mae_discrepancy = mae(theta_sine_reconstructed, theta_sine)
print(' noisy:       {:2.2e}\n denoised:    {:2.2e}\n discrepancy: {:2.2e}\n'.format(mae_noisy, mae_denoised, mae_discrepancy))

### 4 Test discrepancy for increasing number of constraints

In [ ]:
def plot_discrepancy(thetas_sine, thetas_sine_reconstructed,
                     thresh=None, labels=['linear constraints', 'sine constraints']):
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 2.5)
    label = labels[0]
    for n_total in thetas_sine.keys():
        theta_sine = thetas_sine[n_total]
        theta_sine_reconstructed = thetas_sine_reconstructed[n_total]

        mae_noisy = mae(angle_set.theta, theta_noisy)
        mae_denoised = mae(angle_set.theta, theta_sine)
        mae_discrepancy = mae(theta_sine_reconstructed, theta_sine)
        #print(' noisy:{:2.2e} denoised: {:2.2e} discrepancy: {:2.2e}'.format(mae_noisy, mae_denoised, mae_discrepancy))
        if n_total == 0:
            ax.scatter(n_total, mae_discrepancy, color='black')
        elif (thresh is None) or (n_total <= thresh):
            ax.scatter(n_total, mae_discrepancy, color='C0', label=label)
            label = None
        else:
            ax.scatter(n_total, mae_discrepancy, color='C1', label=label)
            label = None
        if n_total == thresh: 
            label = labels[1]
    ax.axvline(n_linear + n_sine, color='black', label='bound')
    #ax.legend(loc='upper right')
    ax.legend()
    #ax.set_ylim(-0.1, 0.7)
    ax.grid()
    ax.set_xlabel('number of constraints')
    ax.set_ylabel('angle discrepancy')
    plt.tight_layout()
    return fig, ax

### first linear then sine constraints

In [ ]:
# required number
from math import floor
from simulation_discrepancy import generate_linear_constraints

Alearn, blearn = generate_linear_constraints(angle_set.points)
print(angle_set.get_n_linear())

Amore, bmore = get_linear_constraints(angle_set, full_rank=False)

A = Afull
b = bfull

num_sine = angle_set.get_n_sine()
all_sine  = int(floor(num_sine * 1.5))
all_linear = A.shape[0] #angle_set.get_n_linear()

thetas_noisy_0 = {}
thetas_sine_0 = {}
thetas_sine_reconstructed_0 = {}
thetas_learned_0 = {}
thetas_learned_reconstructed_0 = {}
for n_linear in range(all_linear + 1):
    if n_linear < all_linear:
        num_sine = 0
    else:
        num_sine = all_sine
        
    for n_sine in range(num_sine + 1):
        if n_sine > 0:
            print('n_sine {}/{}'.format(n_sine, num_sine))
        choices_sine = range(n_sine)
        
        n_total = n_linear + n_sine
        print('n_total', n_total)

        choices_linear = range(n_linear)
        
        # analytic constraints
        theta_sine, __ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                        Afull=A, bfull=b, N=N,
                                                    choices_sine=choices_sine, 
                                                    choices_linear=choices_linear, 
                                                    eps=None)
        theta_sine_reconstructed, __ = reconstruct_theta(theta_sine, angle_set.corners, angle_set.N)
        
        thetas_noisy_0[n_total] = theta_noisy
        thetas_sine_0[n_total] = theta_sine
        thetas_sine_reconstructed_0[n_total] = theta_sine_reconstructed
        
        # learned constraints
        if n_linear <= Alearn.shape[0]:
            theta_learned, __ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                            Afull=Alearn, bfull=blearn, N=N,
                                                        choices_sine=choices_sine, 
                                                        choices_linear=choices_linear)
            theta_learned_reconstructed, __ = reconstruct_theta(theta_learned, angle_set.corners, angle_set.N)
            thetas_learned_0[n_total] = theta_learned
            thetas_learned_reconstructed_0[n_total] = theta_learned_reconstructed

In [ ]:
fig, ax = plot_discrepancy(thetas_sine_0, thetas_sine_reconstructed_0, 
                           thresh=all_linear)
ax.set_title('analytical')

fig, ax = plot_discrepancy(thetas_learned_0, thetas_learned_reconstructed_0, 
                           thresh=all_linear)
ax.set_title('learned')
#savefig('plots/discrepancy.pdf', fig=fig)

### first sine constraints, then linear

In [ ]:
num_sine = angle_set.get_n_sine() # we don't want to slow down too much. 
print(num_sine)
num_all_linear = angle_set.get_n_linear()

Alearn, blearn = generate_linear_constraints(angle_set.points)

thetas_sine_1 = {}
thetas_sine_reconstructed_1 = {}

thetas_learned_1 = {}
thetas_learned_reconstructed_1 = {}

for n_sine in range(num_sine+1):
    print('n_sine', n_sine)
    choices_sine = range(n_sine)
    
    if n_sine < num_sine:
        num_linear = 0
    else:
        num_linear = num_all_linear 
        
    for n_linear in range(num_linear+1):
        if n_linear > 0:
            print('n_linear {}/{}'.format(n_linear, num_linear))
            print('n_total', n_total)
        choices_linear = range(n_linear)
        
        n_total = n_linear + n_sine

        theta_sine,__ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                    Afull=Afull, bfull=bfull, N=N,
                                                    choices_sine=choices_sine, 
                                                    choices_linear=choices_linear)
        theta_sine_reconstructed,__ = reconstruct_theta(theta_sine, angle_set.corners, angle_set.N)
        
        
        theta_learn,__ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                    Afull=Alearn, bfull=blearn, N=N,
                                                    choices_sine=choices_sine, 
                                                    choices_linear=choices_linear)
        theta_learn_reconstructed,__ = reconstruct_theta(theta_learn, angle_set.corners, angle_set.N)

        thetas_sine_1[n_total] = theta_sine
        thetas_sine_reconstructed_1[n_total] = theta_sine_reconstructed
        thetas_learned_1[n_total] = theta_learn
        thetas_learned_reconstructed_1[n_total] = theta_learn_reconstructed

In [ ]:
fig, ax = plot_discrepancy(thetas_sine_1, thetas_sine_reconstructed_1, 
                           thresh=num_sine, 
                           labels=['sine constraints', 'linear constraints'])
fig, ax = plot_discrepancy(thetas_learned_1, thetas_learned_reconstructed_1, 
                           thresh=num_sine, 
                           labels=['sine constraints', 'linear constraints'])
#savefig('plots/discrepancy_sine_first.pdf', fig)

### 5 Create plots for paper

Below results are obtained with simulation_discrepancy.py

In [ ]:
figsize = (3, 4)

# Final submission (run on LCAV server)
ylim = [1e-4, 2e-1]
ylim_err = [1e-9, 2e-3]
fnames = ['discrepancy_ICASSP'] # results non-learning
plotname = 'discrepancy'

#fname1 = 'discrepancy_learned_ICASSP0' # newest results for learning, before interruption at 8, i=14
#fname2 = 'discrepancy_learned_ICASSP1' # newest results for learning, after interruption at 8, i=14
#fnames = [fname1, fname2]
#plotname = 'discrepancy_learned'

# New results
#ylim = None 
#ylim_err = None
#fnames = ['discrepancy']
#plotname = 'discrepancy'
#
#fnames = ['discrepancy_learned'] 
#plotname = 'discrepancy_learned'

In [ ]:
from simulation_discrepancy import mae

dfs = [pd.read_pickle('results/{}.pkl'.format(fname)) for fname in fnames]
df = pd.concat(dfs, ignore_index=True, sort=False)

df.loc[:, 'discrepancy_sine'] = df.apply(lambda row: mae(row.theta_sine, row.theta_sine_reconstructed), axis=1)
df.loc[:, 'discrepancy_noisy'] = df.apply(lambda row: mae(row.theta_noisy, row.theta_noisy_reconstructed), axis=1)
df.loc[:, 'sine'] = df.apply(lambda row: row.n_sine>0, axis=1)
df.tail()

In [ ]:
# plot results
from scipy.special import binom
from helpers import savefig

fig, axs = plt.subplots(2, 1, sharex=True, sharey=False)
fig.set_size_inches(*figsize)
i = 0
Ns = sorted(df.N.unique())

kwargs = dict(
    linewidth=0.0,
    s=20
)
sns.set_palette(sns.color_palette('Blues_d', n_colors=len(Ns)))
legend=False
log=True

label = 0 
for N, df_N in df.groupby('N'):
    N = int(N)
    df_mean = df_N.groupby('n_total').aggregate(np.mean, axis=0)
    df_mean.reset_index(inplace=True, drop=False)
    
    discrepancy_noise=df_mean.discrepancy_noisy.values[0]
    error_noise=df_mean.error_noisy.values[0]
    
    ################ DISCREPANCY PLOT ####################
    ax = axs[0]
    label = 'N = {}'.format(N)
    color = 'C{}'.format(i)
    ax.set_yscale('log')
    sns.scatterplot(data=df_mean, x='n_total', y='discrepancy_sine', ax=ax, 
                    label=label, style='sine', color=color, **kwargs)
    ax.set_xlabel('number of constraints')
    ax.set_ylabel('discrepancy')
    if log:
        ax.set_yscale('log')
        if ylim is not None:
            ax.set_ylim(*ylim)
    
    ################ ERROR PLOT ####################
    ax = axs[1]
    label = None
    ax.set_yscale('log')
    sns.scatterplot(data=df_mean, x='n_total', y='error_sine', 
                    ax=ax, label=label, style='sine', color=color, **kwargs)
    if log:
        ax.axis(yscale='log') 
        if ylim_err is not None:
            ax.set_ylim(*ylim_err)
    ax.set_xlabel('number of constraints')
    ax.set_ylabel('accuracy')
    ax.legend().set_visible(False)
    i += 1

################ FANCY LEGEND ####################
if legend:
    handles, labels = axs[0].get_legend_handles_labels()
    print(labels)
    labels[1] = 'constraint type'
    labels[2] = 'linear'
    labels[3] = 'non-linear'
    order = range(len(labels))[::4]
    ncol = min(len(order), 3)
    l1 = axs[0].legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                       loc='center', ncol=ncol, bbox_to_anchor=[0.5, 1.05], 
                       framealpha=1.0)
    l1.remove()
    order = [2,3] # [1, 2, 3]
    l2 = axs[0].legend([handles[idx] for idx in order],[labels[idx] for idx in order],
                        loc='upper center', ncol=2, bbox_to_anchor=[0.5, -0.0],
                        framealpha=1.0)
    l2.remove()
    # to make sure legend is in foreground.
    ax2 = axs[0].twinx()
    ax2.axis('off')
    ax2.add_artist(l2)
    ax2.add_artist(l1)
else:
    #  Add annotation on top of plots. 
    l1 = axs[0].get_legend()
    l2 = axs[0].get_legend()
    l1.remove()
    l2.remove()
    
    y_offset = -3.8
    y2_offset = -8.4
    xs = [-5.0, 14.0, 42.0, 85.0, 130.0]
    
    ys = [10**(y_offset)]*len(Ns)
    ys[0] = 10**(y_offset+0.3)
    ys2 = [10**(y2_offset)]*len(Ns)
    ys2[0] = 10**(y2_offset+0.7)
    if (plotname != 'discrepancy_new') and (ylim is not None):
        axs[0].get_yaxis().set_ticklabels([])
        axs[1].get_yaxis().set_ticklabels([])
        axs[0].set_ylabel('')
        axs[1].set_ylabel('')
    for j, N in enumerate(Ns):
        axs[0].annotate(s='N={}'.format(int(N)), 
                       xy=(xs[j],ys[j]),
                       color='C{}'.format(j))
        axs[1].annotate(s='N={}'.format(int(N)), 
                       xy=(xs[j],ys2[j]),
                       color='C{}'.format(j))
axs[1].grid()
axs[0].grid()
savefig('plots/{}.pdf'.format(plotname), fig=fig)